# Introduction to GitHub Repo and Gists Reporting

<p>This Notebook provides a basic reporting tool that creates a report based on either a users or an organizations repositories.  </p>
<p>For organizations with a large number of repositories, a Filter List can be used to limit the number of repos returned.</p>
<p>This Notebook also provides a basic reporting tool that creates a report of a users' Gists.  </p>

## About this Notebook
This Notebook uses the following:

* Python 3 
* toc2 nbextension (optional) for displaying Table of Contents.
* MikTex for nbconversion (optional) for converting notebook to pdf

## References and Info
See command below for converting Notebook to web pdf without code cells and prompts
```
jupyter nbconvert .\GitHubReports.ipynb --to webpdf --no-input --output myReport.pdf
```

# GitHub Report
<p>This report is a basic report based on either a users or an organizations GitHub repositories.  </p>

## Prerequisites
<p> This report requires a separate Git credentials file to be placed in same directory as notebook and a reporting strategy (choose eiter user or org)
    </p>

### GitHub Credentials File Setup

<div class="alert alert-block alert-info">
<b>Create security file:</b> 
<p>In the same directory as this Notebook, create a file - gitsecrets.py - and add the following:<br>
    GITHUB_USER= Your Git User Name  <br>
    GITHUB_TOKEN= Your Git Token
</p>
</div>

<div class="alert alert-block alert-warning"><b>GitHub API Limits: </b>You may reach a limit of API calls that GitHub will allow, ex. 2000 calls/minute.  Try to limit the number of repos/folders/files. </div>


## Initialize Report
<p>Required steps before report can be executed</p>

### Import packages and Set Authorization
Credentials file - ***gitsecrets.py*** - needs to be available

In [ ]:
import requests
import pprint
from gitsecrets import GITHUB_USER,GITHUB_TOKEN
from IPython.display import display, Math, HTML, Markdown
import datetime  
import pytz
from dateutil.relativedelta import relativedelta

display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 70em; }</style>"))

# Create auth for Git credentials
auth=requests.auth.HTTPBasicAuth( GITHUB_USER  ,GITHUB_TOKEN )
print('Authorization set up for user: ', GITHUB_USER)
td = datetime.datetime.today()
start_datetime =  td  + relativedelta(days=-7) 
recently_datetime = start_datetime.astimezone(pytz.timezone('US/Eastern'))
print('Recent start date '   ,
      recently_datetime.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d/%Y %I:%M %p %Z") )
# print( + str(recently_datetime))

def string2List (myString):
    out = [] 
    buff = []
    for c in myString:
        if c == '\n' or c == '\r':
            out.append(''.join(buff))
            buff = []
        else:
            buff.append(c)
    else:
        if buff:
           out.append(''.join(buff))
    return out

def show_branch_tree(branchTree,offset):
    x = branchTree
    for tree in x.json()['tree']  :
        if tree['type'] != 'tree':
            print(' '*offset + tree['path']) 
        if tree['type'] == 'tree':
            print(' '*offset +  tree['path'] + '/'       )
            x2 = requests.get(tree['url'] , auth=auth)
            for tree2 in x2.json()['tree']:
                print( ' '*(offset + 4) , tree2['path'] + (  '/'  if tree2['type'] == 'tree' else ''  ) )
                if tree2['type'] == 'tree' :
                    x3 = requests.get(tree2['url'] , auth=auth)
                    for tree3 in x3.json()['tree']:
                        print( ' '*(offset + 8),  tree3['path'] + (  '/'  if tree3['type'] == 'tree' else ''  ) )
                        if tree3['type'] == 'tree' :
                            x4 = requests.get(tree3['url'] , auth=auth)
                            for tree4 in x4.json()['tree']:
                                print( ' '*(offset + 12),  tree4['path'] + (  '/'  if tree4['type'] == 'tree' else ''  ) )
                                if tree4['type'] == 'tree' :
                                    x5 = requests.get(tree4['url'] , auth=auth)
                                    for tree5 in x5.json()['tree']:
                                        print(' '*(offset + 16) ,  tree5['path'] + (  '/'  if tree5['type'] == 'tree' else ''  ) )


### Fetch Repos
You will be prompted for User or Org

In [ ]:
print('Will the report be for a User or Org (User/Org)?')
reportType = input()
org = ''
if reportType == "U" :
    print('Your report will be base on your Git User')
    repoUrl = 'https://api.github.com/user/repos' 
    repoResponse = requests.get(repoUrl, auth=auth)
    print('Authorization Confirmed for user: ' + GITHUB_USER + ' Repo Url is: '+ repoUrl)
elif reportType == "O":
    print('Report will be based on your organization, please type name of org:')
    org = input()
    repoUrl = 'https://api.github.com/orgs/' + org + '/repos'
    repoResponse = requests.get(repoUrl, auth=auth)
    print('Authorization Confirmed for user: ' + GITHUB_USER + ' Repo Url is: '+ repoUrl)
else :
    print ('Error, try again')
reposFilterUrls =[]

## Repos/Filter List 
* Filter List (optional) 
* Repos List (required)

### Create Repo Filter List (optional)
<p>Append name of desired repos to the reposFilterUrls List, as required</p>
Example:  reposFilterUrls.append('myTestRepo')

In [ ]:
tmpReposFilterUrls = []
# Add Filters as required as show below
# tmpReposFilterUrls.append('JupyterCloud')
tmpReposFilterUrls.append('truccr_oci-iac')
tmpReposFilterUrls.append('truccr_oci-ca-dumptruck')
tmpReposFilterUrls.append('truccr_oci-manual-transmission')
# tmpReposFilterUrls.append('truccr_oci-frontloader')
# tmpReposFilterUrls.append('truccr_atlas')

print( 'Number of Filters: '  ,len(tmpReposFilterUrls))
print('Filter List:')
reposFilterUrls = []
for rFilter in tmpReposFilterUrls:
    if type == 'U':
        urlExtn = 'user/repos/'
    else:
        urlExtn = 'repos/' + org + '/'
    newFilter = 'https://api.github.com/' + urlExtn + rFilter.split(":")[0] 
    reposFilterUrls.append(newFilter  )
    print(newFilter)


### Create Repo List (required)
<p>Run cell and verify repo count (Note: max repos is 30)</p>

In [ ]:
repos =[]
# Get Repo object from GitHub
if reposFilterUrls is not None and len(reposFilterUrls) > 0:
    for reposFilterUrl in reposFilterUrls:
        resp = repoResponse = requests.get(reposFilterUrl, auth=auth)
        repos.append(resp.json())
# else:
#     for repoR in repoResponse.json():
#         repos.append(repoR)
print( 'Number of Repos Found: '  ,len(repos))
print('\nRepos in this Report')
for repo in repos:
    print('   '   ,repo['name'])

## Repository Report

In [ ]:
reposIssuesList = []
targetBranch = 'alpha-dev'
# targetBranch = 'main'

for repo in repos :
    repoIssuesDict = {}
    repoIssuesDict['repo_name'] = repo['name']
    print( 'Repo:',repo["name"] + ' and branch: ' + targetBranch )
    print('    Private:     ', repo['private'])
    print('    HTTPS URL:   ', repo['clone_url'])
    print('    SSH URL:     ', repo['ssh_url'])    
    contentsUrl = repo['contents_url']
    print('    Contents URL:' ,contentsUrl.replace('{+path}',''))
    contents = requests.get(contentsUrl.replace('{+path}',''),auth=auth)
    cFileList = []  ;   cDirList = []
    [cFileList.append(x) for x in contents.json() if x['type'] == 'file' ]
#- Files
    print('    Files in main branch:')
    for content in cFileList:
        print('       ', content['name'])
    [cDirList.append(x) for x in contents.json() if x['type'] == 'dir' ]
#- Folders
    print('    Folders in main branch:')
    for content in cDirList:
        print('       ', '/' + content['name'])
#- Branches
    print('    Branches:')
    branches = requests.get(repo['branches_url'].replace('{/branch}',''),auth=auth)
    for branch in branches.json():
        print('         ' ,branch['name'])
        branchContents = requests.get(repo['branches_url'].replace('{/branch}','/'+branch['name'])
                                      ,auth=auth)
        branchContent = branchContents.json()['commit']['commit']
        print('              Author:',branchContent['author'] ['name'])
        print('              Date:  ' ,branchContent ['author'] ['date'] )             
        # Get Branch Url
        if branch['name'] == targetBranch: 
            targetBranchUrl = branch ['commit'] ['url']
#-Folder Contents
    print('\n    Folder Contents in Branch: ' + targetBranch )
    targetBranchContents = requests.get(targetBranchUrl,auth=auth)
    targetBranchTreeUrl = targetBranchContents.json()['commit']['tree'] ['url'] 
#     print('Target Branch URL: ' , targetBranchTreeUrl )
    branchContentTree =requests.get(targetBranchTreeUrl,auth=auth)
    show_branch_tree(branchContentTree,offset=10)
 #- Issues
    print('\n    Open Issues:')
      
    issues = requests.get(repo['issues_url'].replace('{/number}','?state=all'),auth=auth)
    issuesList = sorted(issues.json() ,key=lambda i:i['number']   )
    repoIssuesList  = []
#     try:
    for issue in issuesList :   # issues.json():
        issuesDict = {}
        issuesDict['title'] = issue['title']
        issuesDict['state'] = issue['state']
        issuesDict['created_at'] = issue['created_at']
        issuesDict['closed_at'] = issue['closed_at']
        issuesDict['number'] = issue['number']
        issuesDict['milestone'] = issue['milestone']

        issuesDict['body'] = []
        if 'open' in issue['state']:
            print('\n         ' , '#' + str(issue['number']) + ' - ' +  issue['title'])   
            print('                Created: ' + issue['created_at'])
            print('                Tasks:')
            issueBody = issue['body']
#                 print(issueBody)
            issueTopicsList=[]
            if issueBody is not None:
                issueLineList  = string2List(issueBody)
#                     print(issueLineList)
                for issueLine in issueLineList:
                    if len(issueLine) > 1 and ('[ ]' in  issueLine or '[x]' in issueLine) :
                        issueTopicsList.append(issueLine)
                        print('               ', issueLine)
            else:
                print('                No Tasks' , )   
            issuesDict['body'] = issueTopicsList
        issuesDict['labels'] = []
        if len(issue['labels']) > 0:
            for label in issue['labels'] :
                newLabelDict = {}
                newLabelDict['name']=label['name']
                newLabelDict['description'] = label['description']
                issuesDict['labels'].append(newLabelDict)
        repoIssuesList.append(issuesDict)
    repoIssuesDict['issues'] = repoIssuesList
    print('\n---' )
    reposIssuesList.append(repoIssuesDict)
print('----------')
with open('ReposIssuesList.json', 'w') as convert_file:
     convert_file.write(json.dumps(reposIssuesList))
print('Conversion of Issues List to file is Complete - ReposIssuesList.json'  )

## Milestones

In [ ]:
milestones = ['Multiple Tenancy Capable'  , 'Tachometer Functional' , 'CURB Functional', 'Standards Compliant']
for milestone in milestones:
    print(milestone )
    for repoIssue in reposIssuesList :
        issuesList = sorted(repoIssue['issues'] ,key=lambda i:i['number']   )
        for issue in issuesList :   
            if issue['milestone'] is not None  and  issue['milestone']['title']  == milestone :
                print( '    ' , issue['title'])


In [ ]:
for testI in reposIssuesList[0]['issues']:
    print(testI['title'] + '  ' + testI['number'] )
    
    print('---')
pprint.pprint( reposIssuesList  )

### Report Issues

In [ ]:
# Print issues List created above

issuesList = []
for repo in reposIssuesList:
    for issue in repo['issues']:
        issueDict = {}
        issueDict['repo_name'] = repo['repo_name']
        issueDict['issue_title'] = issue['title']
        issueDict['issue_number'] = issue['number']
        issueDict['date_opened'] = issue['created_at']
        issueDict['date_closed'] = issue['closed_at']
        issueDict['state'] = issue['state']
        issueDict['body'] = issue['body']
        issuesList.append(issueDict)

td = datetime.datetime.today()
start_datetime =  td  + relativedelta(days=-7) 
recently_datetime = start_datetime.astimezone(pytz.timezone('US/Eastern'))
# print('Recent start date '   ,
#       recently_datetime.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d/%Y %I:%M %p %Z") )   
reportTitle = \
"""\hspace {40mm} \Large {\\textbf{TRUCCR Issues Report}} \\\[1pt]
\hspace {60mm} \\small {\\textsf{Date: TDATE  }} """

today = td.astimezone(pytz.timezone('US/Eastern'))
todayDate = today.strftime("%m/%d/%Y")
display(Markdown( '$ ' + reportTitle.replace('TDATE', todayDate  ) + ' $' ))
display(Markdown( '$ \\normalsize \\color{blue}  {\\textsf{Current Open Issues}} \\\[1pt]  $ '  ))
OPEN_ISSUES_TEMPLATE = """
\hspace {10mm} \small {\\textbf{TITLE - #NBR}} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Opened AGE days ago on ODATE }} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Module: MODULE }} \\\[5pt] 
\hspace {10mm} \small {\\textsf{Issue Tasks: }} \\\[1pt] 
\hspace {10mm} \small {TASKS } \\\[1pt] 
"""
ISSUE_TEMPLATE = """
\hspace {10mm} \small {\\textbf{TITLE - #NBR}} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Opened AGE days ago on ODATE }} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Module: MODULE }} \\\[5pt] 
"""
# print ('Open Issues')
for issue in issuesList:
    if issue['state'] == 'open':
#         print('\n' + issue['issue_title'] + ' - #' + str(issue['issue_number'] )   )
        openIssues = OPEN_ISSUES_TEMPLATE
        openIssues = openIssues.replace('TITLE' , issue['issue_title'] )
        openIssues = openIssues.replace('NBR' , str(issue['issue_number']) )
        openedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
        age = td - openedDate
        dateOpened =openedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
#         print('Opened ' + str(age.days) + ' days ago on ' + dateOpened )
#         print('Module: ' + issue['repo_name']    )
        openIssues = openIssues.replace('AGE' , str(age.days ) )
        openIssues = openIssues.replace('ODATE' , dateOpened )
        openIssues = openIssues.replace('MODULE' , str(issue['repo_name']) )
        
        issueBody = issue['body']
        if issueBody is not None :
            issueCnt = 0
#             issueLineList  = string2List(issueBody)
            issueMatrix = ' \\begin{array}{l}  \\\[1pt] '
            for issueLine in issueBody:
                if len(issueLine) > 1 and ('[ ]' in  issueLine or '[x]' in issueLine) :
                    issueCnt += 1
                    matrixLine = ' \\small { \\textsf{LINE }}   \\\[1pt] '
                    issueMatrix = issueMatrix + matrixLine.replace('LINE', issueLine   )
            if issueCnt > 0:
                issueMatrix = issueMatrix + '  \\end{array} \\\[2pt]   '
                openIssues = openIssues.replace('TASKS' , issueMatrix )
            else:
                openIssues = openIssues.replace('TASKS' , '\\textit{No Tasks Entered}' )
        else:
            openIssues = openIssues.replace('TASKS' , '\\textit{No Tasks Entered}' )  
        display(Markdown ('$ ' + openIssues   + ' $'    ))
               
# print('\nIssues Closed Last Week')
display(Markdown( '$ \\normalsize \\color{blue} {\\textsf{Summary Issues Closed Last 7 Days (Including Merged Branches)}} \\\[1pt]  $ '  ))
for issue in issuesList:
    if issue['state'] == 'closed':
        closedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
        dateClosed =closedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
        age = td - closedDate
        if age.days > 6 :# or 'Alpha dev' in issue['issue_title']   :
            continue
        closedIssue = ISSUE_TEMPLATE
        closedIssue = closedIssue.replace('TITLE' , issue['issue_title'] )
        closedIssue = closedIssue.replace('NBR' , str(issue['issue_number']) )            
        closedIssue = closedIssue.replace('AGE' , str(age.days ) )
        closedIssue = closedIssue.replace('ODATE' , dateClosed )
        closedIssue = closedIssue.replace('MODULE' , str(issue['repo_name']) )            
        display(Markdown ('$ ' + closedIssue   + ' $'    ))

# print('\nIssues Opened Last Week')
display(Markdown( '$ \\normalsize \\color{blue} {\\textsf{Summary Issues Opened Last 7 Days }} \\\[1pt]  $ '  ))
for issue in issuesList:
    if issue['state'] == 'open':
        openedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
        age = td - openedDate
        if age.days > 6 :
            continue
#         print('\n' + issue['issue_title'] + ' - #' + str(issue['issue_number'] )   )
        dateOpened =openedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
        
        openIssue = ISSUE_TEMPLATE
        openIssue = openIssue.replace('TITLE' , issue['issue_title'] )
        openIssue = openIssue.replace('NBR' , str(issue['issue_number']) )            
        openIssue = openIssue.replace('AGE' , str(age.days ) )
        openIssue = openIssue.replace('ODATE' , dateOpened )
        openIssue = openIssue.replace('MODULE' , str(issue['repo_name']) )            
        display(Markdown ('$ ' + openIssue   + ' $'    ))       

print('-------------')

### Branches

In [ ]:
for branch in branches.json():
    print('         ' ,branch['name'])
    if branch['name'] == 'alpha-dev':
        branchUrl = branch ['commit'] ['url']
        break
# 
targetBranchContents = requests.get(branchUrl,auth=auth)
targetBranchTreeUrl = targetBranchContents.json()['commit']['tree'] ['url'] 
print('Target Branch URL: ' , targetBranchTreeUrl )

### Comments

In [ ]:
commentsUrl =  requests.get(  myBranchContents.json()['comments_url']  , auth=auth)
pprint.pprint(commentsUrl.json())

# Gists Report

## Import Packages and Fetch Gists

In [ ]:
import requests
import pprint
from gitsecrets import GITHUB_USER,GITHUB_TOKEN
# Create auth for Git credentials
auth=requests.auth.HTTPBasicAuth( GITHUB_USER  ,GITHUB_TOKEN )

userRepoUrl = 'https://api.github.com/gists'
myGistsUrl = userRepoUrl
repoResponse = requests.get(myGistsUrl, auth=auth)

## Gists Report

In [ ]:
gists = repoResponse.json()
for gist in gists:
    print( gist['description'])
    print('    Updated: ', gist['updated_at']  )
    print('    Type:    ', 'Public' if gist['public'] else 'Private' )
    print('    Html Url: ', gist['html_url']  )
    print('    Comments: ', gist['comments']  )
    
    forks = requests.get(gist['forks_url'],auth=auth)
    print('    Forks:    ',str(len(forks.json())))
    files = gist['files']
    print('    Files:')
    for file,extra in files.items():
        print('        File: ',file)
    print('--')
print('\n----------')

# Archives

## Deprecated Main Branch Content Listing

In [ ]:
   for folder in cDirList:
        break
        print('        /'+ folder['name'])
        fContents = requests.get(folder['url'] , auth=auth   )
        cFileList2 = []  ;   cDirList2 = []
        [cFileList2.append(x) for x in fContents.json() if x['type'] == 'file' ]
        for content in cFileList2:
            print('           '+ content['name'])
        [cDirList2.append(x) for x in fContents.json() if x['type'] == 'dir' ]            
        for content in cDirList2:
            print('           /'+ content['name'])     
            folder2 = folder['name'] + '/' + content['name']
#             print(contentsUrl.replace('{+path}',folder2 ))
            fContents2 = requests.get(contentsUrl.replace('{+path}',folder2),auth=auth)
            cFileList3 = []  ;   cDirList3 = []
            [cFileList3.append(x) for x in fContents2.json() if x['type'] == 'file' ]    
            for content2 in cFileList3:
                print('               ', content2['name'] )
            [cDirList3.append(x) for x in fContents2.json() if x['type'] == 'dir' ] 
            for content2 in cDirList3:
                print('               /'+ content2['name'] )      
                newDirName =  folder['name']  +'/'+content['name'] + '/' + content2['name']
                fContents3 = requests.get(contentsUrl.replace('{+path}',newDirName),auth=auth)
                cFileList4 = []  ;   cDirList4 = []
                [cFileList4.append(x) for x in fContents3.json() if x['type'] == 'file' ]    
                for content3 in cFileList4:
                    print('                   ', content3['name'] )
                [cDirList4.append(x) for x in fContents3.json() if x['type'] == 'dir' ]
                for content3 in cDirList4:
                    print('                   /'+ content3['name'] )   
                    newDirName2 =  folder['name']  +'/'+content['name'] + '/' + content2['name'] \
                                 + '/' + content3['name']
                    fContents4 = requests.get(contentsUrl.replace('{+path}',newDirName2),auth=auth)
                    cFileList5 = []  ;   cDirList5 = []
                    [cFileList5.append(x) for x in fContents4.json() if x['type'] == 'file' ]    
                    for content4 in cFileList5:
                        print('                       ', content4['name'] )
                    [cDirList5.append(x) for x in fContents4.json() if x['type'] == 'dir' ]
                    for content4 in cDirList5:
                        print('                   /'+ content4['name'] )   

## Issues

In [ ]:


searchFilter = 'https://api.github.com/repos/tr/truccr_oci-iac/issues?state=all'
repoResponse = requests.get(searchFilter, auth=auth)
closedIssues = repoResponse.json()

# pprint.pprint(closedIssues[i].keys() )
for i in range(len(closedIssues)):
    createdTime = closedIssues[i]['created_at'] 
    closedTime = closedIssues[i]['closed_at'] 
    if 'open' == closedIssues[i]['state'] :
        newTime = datetime.datetime.strptime(createdTime,"%Y-%m-%dT%H:%M:%SZ")
        age = td - newTime
        print('opened ' + str(age.days) + ' days ago'  )
    else:
        newTime = datetime.datetime.strptime(closedTime,"%Y-%m-%dT%H:%M:%SZ")
    newTime = newTime.astimezone(pytz.timezone('US/Eastern'))
    if newTime > recently_datetime :
        print('Recently Opened/Closed')
    print(closedIssues[i]['title'] )
    print(closedIssues[i]['state'] )
    print(newTime.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d/%Y %I:%M %p %Z") )
    print('---')
#     print(closedIssues[i]['repository']    )